# Collecte, Traitement, et Analyse de données de réseaux sociaux

Importation des librairies

In [5]:
import json
import re
import pandas as pd
from textblob import TextBlob

Ouverture du fichier Json et nettoyage du text

In [ ]:
#Ouverture du fichier jason
with open('versailles_tweets_100.json') as json_data:
    data_dict = json.load(json_data)

#Effacer tous les caractères spéciaux dans le texte

for dict in data_dict:
    #print (dict["text"], end = '\n')
    dict["text"] = re.sub(r'[\W_]+',' ', dict["text"])
    print(dict["text"])
    
# Création et écriture dans le fichier zone d'atterissage
with open("zone_d'atterrissage.json", 'w') as f2:
    json.dump(data_dict, f2)



Identification de l'auteur de la publication

In [ ]:
with open('versailles_tweets_100.json') as json_data:
    data = json.load(json_data)

liste_auteurs= []
for dict in data:
    liste_auteurs.append(dict['_id'])
print(liste_auteurs)

Extraction de la liste des hashtags et la liste des mentions de chaque publication

In [ ]:
liste_hashtags = []
liste_mentions = []

for dict in data:
    #Extraction des hashtags
    hashtags = re.findall(r'\B#\w*[a-zA-Z]+\w*', dict['text'])
    liste_hashtags.append(hashtags)
    #Extraction des mentions
    mentions = re.findall(r'\B@\w*[a-zA-Z]+\w*', dict['text'])
    liste_mentions.append(mentions)

print(liste_hashtags)

print(liste_mentions)

Analyse de sentiments de la publication

In [61]:
liste_sentiments = []
with open("zone_d'atterrissage.json") as f:
   clean_data  = json.load(f)

for dico in clean_data:
    sentiment = TextBlob(dico['text']).sentiment 
    liste_sentiments.append(sentiment[1])
print(liste_sentiments)


[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Identification du/des topics de la publication 

Création du DataFrame

In [ ]:
df = pd.DataFrame(list(zip(liste_auteurs,liste_hashtags, liste_mentions, liste_sentiments)), columns = ['ID', 'Hashtags', 'Mentions', 'Sentiments'])
df

Top K hashtags

In [ ]:
# création d'un dictionnaire contenant un hashtag et le nombre de fois qu'il apparaît
def K_hashtags(liste_hashtags):
    occurences = []
    dic = {}
    for i in range (len(liste_hashtags)):
            dic[liste_hashtags[i]] = liste_hashtags.count(liste_hashtags[i]) 
    return dic

K_hashtags(liste_hashtags)


Top K utilisateurs mentionnés

In [ ]:
# création d'un dictionnaire contenant un utilisateur et le nombre de fois qu'il est mentionné

def K_mention(liste_mentions):
    occurences = []
    dic = {}
    for i in range (len(liste_mentions)):
            dic[liste_mentions[i]] = liste_mentions.count(liste_mentions[i]) 
    return dic

K_mention(liste_mentions)


Top K utilisateur

In [ ]:
# création d'un dictionnaire contenant un utilisateur et le nombre de fois qu'il a posté un tweet
def K_utilisateur(liste_auteurs):
    occurences = []
    dic = {}
    for i in range (len(liste_auteurs)):
            dic[liste_auteurs[i]] = liste_auteurs.count(liste_auteurs[i]) 
    return dic

K_utilisateur(liste_auteurs)